In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import urllib.request
from fake_useragent import UserAgent

In [2]:
def get_url(page):
       
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.maximize_window()
        driver.get(f"https://www.wildberries.ru/catalog/sport/vidy-sporta/fitnes/trenazhery?page={page}&sort=popular&xsubject=4298")
        unique_items = set()
        while True:
            item = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, 'product-card__link.j-card-link.j-open-full-product-card')))
            H = driver.execute_script('return document.body.scrollHeight;')
            h = 0
            while True :
                h += 300
                if h >= H :
                    break

                for element in WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'product-card__link.j-card-link.j-open-full-product-card'))):
                    if element.get_attribute('href') not in unique_items:
                        yield element.get_attribute('href')
                    unique_items.add(element.get_attribute('href'))
                driver.execute_script("window.scrollTo(0, {});".format(h))
                time.sleep(0.1)
            try:
                button = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,"//button[contains(@class,'Buttonreact__StyledButton')][./i[contains(.,'arrow_forward')]]")))
                driver.execute_script('arguments[0].click();',button)
                WebDriverWait(driver,3).until(EC.staleness_of(item))
            except Exception as e:
                return
        driver.quit()


In [3]:
def get_item(page):
    for url_item in get_url(page):

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.maximize_window()
        driver.get(url_item)


        driver.implicitly_wait(3)

        try:
            title = driver.find_element(By.TAG_NAME, 'h1').text
        except:
            title = ''

        article = ''

        try:
            price = driver.find_elements(By.CLASS_NAME, 'price-block')
            price_lst = []
            for p in price:
                if p.text != '':
                    price_lst.append(p.text)
            price_fin = price_lst


        except:
            price_fin = ''


        try:
            cat = driver.find_element(By.CLASS_NAME, 'breadcrumbs__list').text.split('\n')
            try:
                cat_1 = cat[-3].strip()
            except:
                cat_1 = ''
            try:
                cat_2 = cat[-2].strip()
            except:
                cat_2 = ''
            try:
                cat_3 = cat[-1].strip()
            except:
                cat_3 = ''
        except:
            cat = ''
        try:
            link = driver.find_element(By.CLASS_NAME, 'photo-zoom__preview.j-zoom-image.hide')
            url = link.get_attribute('src')
        except:
            url = ''

        try:
            WebDriverWait(driver,8).until(EC.presence_of_element_located((By.CLASS_NAME, "collapsible__toggle.j-parameters-btn.j-wba-card-item.j-wba-card-item-show"))).click()
        except:
            pass

        desc = ''

        try:
            char = driver.find_elements(By.CLASS_NAME, 'product-params')
            char_list = []
            for ch in char:
                if ch.text != '':
                    char_list.append(ch.text.replace('\n', ' '))
            chars = ' '.join(char_list)

        except:
            chars = ''

        driver.quit()

        yield title, article, price_fin, cat_1, cat_2, cat_3, desc, chars, url
       

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3', 
              'description', 'caracteristics', 'url']

df = pd.DataFrame(columns = df_columns)

In [5]:
def to_csv(page):

    for item in get_item(page):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
           
    df.to_csv('wb.csv')

In [6]:
for i in range(1, 5):
    to_csv(i)
    print(f'Log: page {i} saved...')

Log: page 1 saved...
Log: page 2 saved...
Log: page 3 saved...
Log: page 4 saved...


In [7]:
df

,title,article,price,cat_1,cat_2,cat_3,description,caracteristics,url
0,Беговая дорожка для дома складная механическая...,,[7 042 ₽\n18 780 ₽\nКредит от\n704 ₽],Фитнес и тренажеры,Тренажеры,BRADEX,,Габариты Глубина предмета 110 см Ширина предме...,https://basket-01.wb.ru/vol59/part5916/5916588...
1,Беговая дорожка для дома ST-310 / складная / э...,,[17 167 ₽\n57 225 ₽\nКредит от\n1 716 ₽],Фитнес и тренажеры,Тренажеры,UNIXFIT,,Дополнительная информация Максимальный вес пол...,https://basket-04.wb.ru/vol543/part54333/54333...
2,Беговая дорожка для дома электрическая компакт...,,[39 296 ₽\n52 395 ₽],Фитнес и тренажеры,Тренажеры,Orlauf,,Дополнительная информация Регулировка угла нак...,https://basket-12.wb.ru/vol1714/part171428/171...
3,Беговая дорожка TM-M50 тренажер,,[12 600 ₽\n21 001 ₽\nКредит от\n1 260 ₽],Фитнес и тренажеры,Тренажеры,Planta,,Габариты Вес без упаковки (кг) 10.5 кг Ширина ...,https://basket-05.wb.ru/vol819/part81945/81945...
4,Беговая дорожка для дома электрическая,,[26 917 ₽\n71 780 ₽\nКредит от\n2 691 ₽],Фитнес и тренажеры,Тренажеры,Unix,,Дополнительная информация Максимальная скорост...,https://basket-12.wb.ru/vol1684/part168491/168...
...,...,...,...,...,...,...,...,...,...
115,Беговая дорожка R-200V,,[18 367 ₽\n69 972 ₽],Фитнес и тренажеры,Тренажеры,UNIXFIT,,Габариты Вес без упаковки (кг) 19 кг Ширина пр...,https://basket-03.wb.ru/vol343/part34349/34349...
116,Беговая дорожка магнитная BT-2760B-H,,[17 992 ₽\n68 543 ₽\nКредит от\n1 799 ₽],Фитнес и тренажеры,Тренажеры,Body Sculpture,,Дополнительная информация Страна производства ...,https://basket-12.wb.ru/vol1716/part171663/171...
117,R-200/беговая дорожка/электрическая/тренажеры ...,,[17 917 ₽\n68 258 ₽\nКредит от\n1 791 ₽],Фитнес и тренажеры,Тренажеры,Unix Fit,,Габариты Глубина предмета 124 см Вес без упако...,https://basket-04.wb.ru/vol488/part48844/48844...
118,Беговая дорожка электрическая GB-1140,,[17 842 ₽\n67 972 ₽\nКредит от\n1 784 ₽],Фитнес и тренажеры,Тренажеры,SportElite,,Дополнительная информация Страна производства ...,https://basket-12.wb.ru/vol1716/part171682/171...


In [8]:
def show_image(val):
    return '<a href="{}"><img src="{}" width=10000></img></a>'.format(val, val)
df.iloc[44:46].style.format({'url': show_image, **{'width': '100px'}})

,title,article,price,cat_1,cat_2,cat_3,description,caracteristics,url
44,Беговая дорожка домашняя OXYGEN New Classic Aurum TFT,,['116 992 ₽\n155 990 ₽\nКредит от\n11 699 ₽'],Фитнес и тренажеры,Тренажеры,Oxygen Fitness,,Дополнительная информация Страна производства Китай Тип беговой дорожки электрическая; домашняя; для дома Количество программ тренировки 82 Регулировка угла наклона да Ширина бегового полотна 50 см Максимальный вес пользователя 150 кг Максимальная скорость дорожки 20; 20 км/ч Максимальный угол наклона 20 градусов Общие характеристики Модель Беговая дорожка для дома Oxygen New Classic Aurum TFT; Беговая дорожка электрическая Oxygen New Classic Aurum TFT Гарантийный срок 1 год Габариты Ширина предмета 88 см Вес без упаковки (кг) 97 кг Вес с упаковкой (кг) 112 кг Высота предмета 149 см Глубина предмета 190 см Длина упаковки 230 см Высота упаковки 33 см Ширина упаковки 92 см Технические особенности Мощность (лс) 3.5 лс,
45,Беговая дорожка домашняя Oxygen New Classic Aurum AC LCD,,['109 492 ₽\n145 990 ₽'],Фитнес и тренажеры,Тренажеры,Oxygen Fitness,,Габариты Вес с упаковкой (кг) 112 кг Ширина предмета 88 см Вес без упаковки (кг) 98 кг Высота предмета 32.5 см Глубина предмета 190 см Длина упаковки 230 см Высота упаковки 33 см Ширина упаковки 91 см Дополнительная информация Ширина бегового полотна 50 см Максимальный угол наклона 20 градусов Максимальный вес пользователя 150 кг Тип беговой дорожки электрическая; домашняя; для дома Максимальная скорость дорожки 20; 20 км/ч Регулировка угла наклона да Страна производства Китай Количество программ тренировки 28 Технические особенности Мощность (лс) 3.5 лс Общие характеристики Модель Беговая дорожка для дома Oxygen New Classic Aurum AC LCD; Беговая дорожка электрическая Oxygen New Classic Aurum AC LCD Гарантийный срок 1 год,


In [22]:
df['price'] = df['price'].apply(lambda x: ''.join(x).split('\n')[0].replace(' ','').replace('₽', ''))
df

,title,article,price,cat_1,cat_2,cat_3,description,caracteristics,url
0,Беговая дорожка для дома складная механическая...,,7042,Фитнес и тренажеры,Тренажеры,BRADEX,,Габариты Глубина предмета 110 см Ширина предме...,https://basket-01.wb.ru/vol59/part5916/5916588...
1,Беговая дорожка для дома ST-310 / складная / э...,,17167,Фитнес и тренажеры,Тренажеры,UNIXFIT,,Дополнительная информация Максимальный вес пол...,https://basket-04.wb.ru/vol543/part54333/54333...
2,Беговая дорожка для дома электрическая компакт...,,39296,Фитнес и тренажеры,Тренажеры,Orlauf,,Дополнительная информация Регулировка угла нак...,https://basket-12.wb.ru/vol1714/part171428/171...
3,Беговая дорожка TM-M50 тренажер,,12600,Фитнес и тренажеры,Тренажеры,Planta,,Габариты Вес без упаковки (кг) 10.5 кг Ширина ...,https://basket-05.wb.ru/vol819/part81945/81945...
4,Беговая дорожка для дома электрическая,,26917,Фитнес и тренажеры,Тренажеры,Unix,,Дополнительная информация Максимальная скорост...,https://basket-12.wb.ru/vol1684/part168491/168...
...,...,...,...,...,...,...,...,...,...
115,Беговая дорожка R-200V,,18367,Фитнес и тренажеры,Тренажеры,UNIXFIT,,Габариты Вес без упаковки (кг) 19 кг Ширина пр...,https://basket-03.wb.ru/vol343/part34349/34349...
116,Беговая дорожка магнитная BT-2760B-H,,17992,Фитнес и тренажеры,Тренажеры,Body Sculpture,,Дополнительная информация Страна производства ...,https://basket-12.wb.ru/vol1716/part171663/171...
117,R-200/беговая дорожка/электрическая/тренажеры ...,,17917,Фитнес и тренажеры,Тренажеры,Unix Fit,,Габариты Глубина предмета 124 см Вес без упако...,https://basket-04.wb.ru/vol488/part48844/48844...
118,Беговая дорожка электрическая GB-1140,,17842,Фитнес и тренажеры,Тренажеры,SportElite,,Дополнительная информация Страна производства ...,https://basket-12.wb.ru/vol1716/part171682/171...


In [23]:
df.to_csv('wb.csv')